# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data = "../Output Data File/output_data.csv"

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_wx = pd.read_csv(output_data)
city_wx.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,praia da vitoria,38.7333,-27.0667,62.01,94,40,12.73,PT,1615084287
1,1,samusu,-33.6500,115.3333,73.00,76,13,1.99,AU,1615084248
2,2,albany,42.6001,-73.9662,23.00,55,3,1.01,US,1615084201
3,3,avarua,-21.2078,-159.7750,78.80,94,75,14.97,CK,1615084339
4,4,hami,42.8000,93.4500,43.74,53,0,5.91,CN,1615084339


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
gmaps.configure(api_key=g_key)

In [10]:
locations = city_wx[["Lat","Lng"]]
humidity = city_wx["Humidity"]

heatmap = gmaps.figure(center = [0,0], zoom_level = 2.0)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(humidity), point_radius = 2.8,)

heatmap.add_layer(heat_layer)

heatmap

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
